In [ ]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from dotenv import load_dotenv
load_dotenv()

print(os.environ.get('FRED_API_KEY'))
print(os.environ.get('FMP_API_KEY'))
print(os.environ.get('NASDAQ_API_KEY'))

import quandl

def resample_from_monthly_factor_series(series):
    monthly_mean = series.resample('MS').mean()  
    monthly_change = monthly_mean.pct_change()
    quarterly_mean = series.resample('QS').mean()  
    quarterly_change = quarterly_mean.pct_change()
    yearly_mean = series.resample('YS').mean()  
    yearly_change = yearly_mean.pct_change()
    
    return {
        'Monthly': monthly_mean, 
        'Monthly Change': monthly_change, 
        'Quarterly': quarterly_mean, 
        'Quarterly Change': quarterly_change, 
        'Yearly': yearly_mean, 
        'Yearly Change': yearly_change
    }


us_m2_data = quandl.get("FED/M2_N_M", authtoken=os.environ.get('NASDAQ_API_KEY'))
print(f'head: {us_m2_data.head()}')
print(f'tail: {us_m2_data.tail()}')
print(f'description: {us_m2_data.describe()}')
    
us_m2_data = us_m2_data.shift(1, freq='D')  # shift the index by 1 day to align with FRED data reporting
print(f'head: {us_m2_data.head()}')
print(f'tail: {us_m2_data.tail()}')
print(f'description: {us_m2_data.describe()}')

us_m2_dict = resample_from_monthly_factor_series(us_m2_data['Value'])

In [ ]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from datetime import datetime, timedelta

import pandas as pd
from fredapi import Fred

def resample_from_monthly_factor_series(series):
    monthly_mean = series.resample('MS').mean()  
    monthly_change = monthly_mean.pct_change()
    quarterly_mean = series.resample('QS').mean()  
    quarterly_change = quarterly_mean.pct_change()
    yearly_mean = series.resample('YS').mean()  
    yearly_change = yearly_mean.pct_change()
    
    return {
        'Monthly': monthly_mean, 
        'Monthly Change': monthly_change, 
        'Quarterly': quarterly_mean, 
        'Quarterly Change': quarterly_change, 
        'Yearly': yearly_mean, 
        'Yearly Change': yearly_change
    }

def resample_from_quarterly_factor_series(series):
    quarterly_mean = series.resample('QS').mean()
    quarterly_change = quarterly_mean.pct_change()
    yearly_mean = series.resample('YS').mean()
    yearly_change = yearly_mean.pct_change()
    
    return {
        'Quarterly': quarterly_mean, 
        'Quarterly Change': quarterly_change, 
        'Yearly': yearly_mean, 
        'Yearly Change': yearly_change
    }

def get_historical_macro_data(start_date, end_date):
    rate_series = ['FEDFUNDS', 'T5YIFR']
    monthly_series_names = {
        'FEDFUNDS': 'Federal Funds Rate',
        'UNRATE': 'Unemployment Rate',
        'CPIAUCSL': 'CPI',
        'PCE': 'PCE',
        'RSAFS': 'Retail Sales',
        'ICSA': 'Initial Claims',
        'HOUST': 'Housing Starts',
        'T5YIFR': '5-Year Forward Inflation Expectation Rate',
        'USEPUINDXD': 'Economic Policy Uncertainty Index for United States',
        'GS10': '10-Year Treasury Constant Maturity Rate'
    }

    quarterly_series_names = {
        'GDPC1': 'GDP',
    }

    fred = Fred(api_key=os.environ.get('FRED_API_KEY'))
    
    macro_data_dict = {}
    for series_code, series_name in monthly_series_names.items():
        series = fred.get_series(series_code, start_date, end_date)
        
        if series_code in rate_series:
            series = series / 100  # convert from decimal to percentage

        macro_data_dict[series_name] = resample_from_monthly_factor_series(series)

    for series_code, series_name in quarterly_series_names.items():
        series = fred.get_series(series_code, start_date, end_date)

        macro_data_dict[series_name] = resample_from_quarterly_factor_series(series)

    us_m2_money_supply_base = quandl.get("FED/M2_N_M", authtoken=os.environ.get('NASDAQ_API_KEY'), start_date=start_date, end_date=end_date)
    us_m2_money_supply_base = us_m2_money_supply_base.shift(1, freq='D')  # shift the index by 1 day to align with FRED data reporting

    macro_data_dict['US M2 Money Supply'] = resample_from_monthly_factor_series(us_m2_money_supply_base['Value'])

    return macro_data_dict


start_date = '2014-01-01'
end_date = datetime.now() - timedelta(1)

# get the data
macro_data_dict = get_historical_macro_data(start_date, end_date)


In [ ]:
# plot the raw data
import plotly.graph_objs as go

def plot_historical_macro_data(factor, time_basis, time_series):
    plots = []

    fig = go.Figure()
    # Add macroeconomic factor trace
    fig.add_trace(
        go.Scatter(x=time_series.index, y=time_series, mode='lines', name=factor))

    if 'Change' in factor or 'Rate' in factor:            
        fig.update_layout(
            title=f'{factor} Historical Data<br><sup>({time_basis})</sup>',
            yaxis_tickformat='.1%'
        )
    else:
        # TODO: need a lookup by factor for what the units are on the y-axis
        fig.update_layout(
            title=f'{factor} Historical Data<br><sup>({time_basis})</sup>',
        )

    return fig

for factor in macro_data_dict.keys():
    print(f'plotting {factor} data')
    for time_basis in macro_data_dict[factor].keys():
        fig = plot_historical_macro_data(factor, time_basis, macro_data_dict[factor][time_basis])
        fig.show()
        
print('*** done plotting raw data ***')

In [ ]:
macro_data_dict['PCE']['Monthly'].describe()


In [ ]:
import altair as alt
import pandas as pd

# Define the desired order of series
series_order = ['Monthly', 'Monthly Change', 'Quarterly', 'Quarterly Change', 'Yearly', 'Yearly Change']

charts = []  # a list to hold the charts for each factor

for factor, data_dict in macro_data_dict.items():
    # Combine all series for a single factor into a DataFrame for plotting
    factor_df = pd.DataFrame()

    # Sort series according to the desired order
    ordered_data_dict = {key: data_dict[key] for key in series_order if key in data_dict}

    for series_name, series in ordered_data_dict.items():
        temp_df = pd.DataFrame(series)
        temp_df = temp_df.reset_index()
        temp_df.columns = ['Date', 'Value']
        temp_df['Series'] = series_name
        factor_df = pd.concat([factor_df, temp_df], axis=0)

    # Create a chart for the factor and add it to the list
    chart = alt.Chart(factor_df, title=factor).mark_line().encode(
        x='Date:T',
        y=alt.Y('Value:Q', scale=alt.Scale(zero=False)),
        facet=alt.Facet('Series:N', columns=len(ordered_data_dict))
    ).properties(
        width=200,
        height=200
    ).resolve_scale(
        y='independent'
    )

    charts.append(chart)

# Combine the charts vertically
final_chart = alt.vconcat(*charts, spacing=20)

final_chart

In [ ]:
import glob
import itertools
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from fredapi import Fred
import quandl
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# disable prophet logging
import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").disabled=True


def tune_hyperparameters(df, initial, period, horizon):
    print(f'initial_window_in_days: {initial}, period_in_days: {period}, horizon_in_days: {horizon}')
    print(f'df.shape: {df.shape}\ndf.head():\n{df.head()}')
    
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
    }
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    smapes = []  # Store the sMAPEs for each params here
    coverages = []  # Store the Coverages for each params here
    
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial=f'{initial} days', period=f'{period} days', horizon=f'{horizon} days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        smapes.append(df_p['smape'].values[0])
        coverages.append(df_p['coverage'].values[0])

    # Find the best parameters
    best_params_rmse = all_params[np.argmin(rmses)]
    best_params_smape = all_params[np.argmin(smapes)]
    best_params_coverage = all_params[np.argmax(coverages)]

    return best_params_rmse, best_params_smape, best_params_coverage

# horizon = number of days to forecast - when only horizon is specified, prophet defaults initial to 3x horizon, period to be 1/2 horizon
# initial = number of days to train on
# period = number of days between training points - used to calculate cutoffs, default is 1/2 horizon
def tune_hyperparameters_for_macro_factors(macro_data_dict, initial_window_in_days, period_in_days, horizon_in_days):
    
    # Define a DataFrame to store the tuned hyperparameters for each time series
    # Set 'time_series_name' and 'macro_factor', e.g., Month over Month, CPI Monthly Change, as the index for easier lookup
    rmse_optimized_df = pd.DataFrame(columns=['rating', 'time_basis', 'macro_factor', 'rmse', 'initial_window', 'period', 'horizon', 'changepoint_prior_scale', 'seasonality_prior_scale', 'seasonality_mode'])
    smape_optimized_df = pd.DataFrame(columns=['rating', 'time_basis', 'macro_factor', 'mape', 'initial_window', 'period', 'horizon', 'changepoint_prior_scale', 'seasonality_prior_scale', 'seasonality_mode'])
    coverage_optimized_df = pd.DataFrame(columns=['rating', 'time_basis', 'macro_factor', 'mape', 'initial_window', 'period', 'horizon', 'changepoint_prior_scale', 'seasonality_prior_scale', 'seasonality_mode'])
        
    # Loop over the time series data
    for factor, time_bases in macro_data_dict.items():
        for time_basis, time_series in time_bases.items():
            
            df = time_series.to_frame()
            df.columns = ['y']
            df['ds'] = df.index
            df.dropna(inplace=True)
            
            # Tune hyperparameters
            best_rmse, best_mape, best_coverage = tune_hyperparameters(df, initial_window_in_days, period_in_days, horizon_in_days)
           
            # Store the tuned hyperparameters - add a rating column for later analysis
            rmse_optimized_df = rmse_optimized_df.append({'rating': "n/a/", 'time_basis': time_basis, 'macro_factor': factor, 'rmse': best_rmse, 'initial_window': initial_window_in_days, 'period': period_in_days, 'horizon': horizon_in_days, 'changepoint_prior_scale': best_rmse['changepoint_prior_scale'], 'seasonality_prior_scale': best_rmse['seasonality_prior_scale'], 'seasonality_mode': best_rmse['seasonality_mode']}, ignore_index=True)
            smape_optimized_df = smape_optimized_df.append({'rating': "n/a/",'time_basis': time_basis, 'macro_factor': factor, 'mape': best_mape, 'initial_window': initial_window_in_days, 'period': period_in_days, 'horizon': horizon_in_days, 'changepoint_prior_scale': best_mape['changepoint_prior_scale'], 'seasonality_prior_scale': best_mape['seasonality_prior_scale'], 'seasonality_mode': best_mape['seasonality_mode']}, ignore_index=True)
            coverage_optimized_df = coverage_optimized_df.append({'rating': "n/a/",'time_basis': time_basis, 'macro_factor': factor, 'mape': best_coverage, 'initial_window': initial_window_in_days, 'period': period_in_days, 'horizon': horizon_in_days, 'changepoint_prior_scale': best_coverage['changepoint_prior_scale'], 'seasonality_prior_scale': best_coverage['seasonality_prior_scale'], 'seasonality_mode': best_coverage['seasonality_mode']}, ignore_index=True)

    rmse_optimized_df.set_index(['macro_factor','time_basis'], inplace=True)
    smape_optimized_df.set_index(['macro_factor','time_basis'], inplace=True)
    coverage_optimized_df.set_index(['macro_factor','time_basis'], inplace=True)
    
    return rmse_optimized_df, smape_optimized_df, coverage_optimized_df

def save_tuned_hyperparameters(tuned_params_df, suffix=None):
    # Save tuned hyperparameters to a csv file
    date = datetime.now().strftime("%Y%m%d")
   # print(f'date: {date}')
    
    file_name = f'tuned_macro_hyperparameters_{suffix}_{date}'
    
    with open(file_name, 'w') as f:
        tuned_params_df.to_csv(f)
        
    return

def load_latest_tuned_hyperparameters():
    # List all files that begin with "tuned_macro_hyperparameters"
    files = glob.glob('tuned_macro_hyperparameters_*.csv')

    # If no files found, return None
    if not files:
        return None

    # Sort files by date
    files.sort(key=os.path.getmtime, reverse=True)

    # Load the most recent file
    latest_file = files[0]
    df = pd.read_csv(latest_file)

    return df

def load_tuned_hyperparameters(file_name):

    df = pd.read_csv(file_name)

    return df

initial_window_in_days = 365*6
horizon_in_days = 365 
period_in_days = 180 

# for month to month data, we want to forecast 1 month ahead and do that for N periods
# for quarter to quarter data, we want to forecast 1 quarter ahead and do that for N periods
# for year to year data, we want to forecast 1 year ahead and do that for N periods

# initial window will impact training model

rmse_optimized_df, smape_optimized_df, coverage_optimized_df = tune_hyperparameters_for_macro_factors(macro_data_dict, initial_window_in_days, period_in_days, horizon_in_days)

suffix = f'rmse_init_{initial_window_in_days}_period_{period_in_days}_horizon_{horizon_in_days}'
save_tuned_hyperparameters(rmse_optimized_df, suffix)

suffix = f'coverage_init_{initial_window_in_days}_period_{period_in_days}_horizon_{horizon_in_days}'
save_tuned_hyperparameters(coverage_optimized_df, suffix)

suffix = f'smape_init_{initial_window_in_days}_period_{period_in_days}_horizon_{horizon_in_days}'
save_tuned_hyperparameters(smape_optimized_df, suffix)


macro_factor in ['US Money Supply', 'CPI', 'PCE', etc.]
time_basis in ['Monthly', 'Monthly Change', 'Quarterly', 'Yearly', etc.]
hyper_parameter_optimization_type in ['rmse', 'smape', 'coverage']
hyper_parameters {
    'changepoint_prior_scale': float, 
    'seasonality_prior_scale': float, 
    'seasonality_mode'in ['additive', 'multiplicative'],
}

hyper_parameter_dict = {}
hyper_parameter_dict['CPI']['Quarterly'][rmse]['ranking']
hyper_parameter_dict['CPI']['Quarterly'][rmse]['seasonality_prior_scale']

{'macro_factor': {
    'US Money Supply': {
        'Monthly': {
            'rmse':
                {'ranking': str, 'changepoint_prior_scale': float, 'seasonality_prior_scale': float, 'seasonality_mode': 'multiplicative'},
            'coverage':
                {'ranking': str, 'changepoint_prior_scale': float, 'seasonality_prior_scale': float, 'seasonality_mode': 'multiplicative'},
            },
    'CPI': {
        'Monthly': {
            'rmse':
                {'ranking': str, 'changepoint_prior_scale': float, 'seasonality_prior_scale': float, 'seasonality_mode': 'multiplicative'},
            'coverage':
                {'ranking': str, 'changepoint_prior_scale': float, 'seasonality_prior_scale': float, 'seasonality_mode': 'multiplicative'},
            },
    }
  }
}

In [ ]:
import itertools
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import json
import glob

# disable prophet logging
import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").disabled=True


def tune_hyperparameters(df, initial, period, horizon):
    print(f'initial_window_in_days: {initial}, period_in_days: {period}, horizon_in_days: {horizon}')
    print(f'df.shape: {df.shape}\ndf.head():\n{df.head()}')
    
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
    }
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    smapes = []  # Store the sMAPEs for each params here
    coverages = []  # Store the Coverages for each params here
    
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial=f'{initial} days', period=f'{period} days', horizon=f'{horizon} days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        smapes.append(df_p['smape'].values[0])
        coverages.append(df_p['coverage'].values[0])

    # Find the best parameters
    best_params_rmse = all_params[np.argmin(rmses)]
    best_params_smape = all_params[np.argmin(smapes)]
    best_params_coverage = all_params[np.argmax(coverages)]

    return best_params_rmse, best_params_smape, best_params_coverage

def tune_hyperparameters_for_macro_factors(macro_data_dict, initial_window_in_days, period_in_days, horizon_in_days):
    # Dictionary to store the hyperparameters
    hyper_parameter_dict = {}

    # Loop over the time series data
    for factor, time_bases in macro_data_dict.items():
        hyper_parameter_dict[factor] = {}
        for time_basis, time_series in time_bases.items():
            hyper_parameter_dict[factor][time_basis] = {}
            
            df = time_series.to_frame()
            df.columns = ['y']
            df['ds'] = df.index
            df.dropna(inplace=True)
            
            # Tune hyperparameters
            best_rmse, best_mape, best_coverage = tune_hyperparameters(df, initial_window_in_days, period_in_days, horizon_in_days)
           
            # Store the tuned hyperparameters
            hyper_parameter_dict[factor][time_basis]['rmse'] = {'rating': 'n/a', **best_rmse}
            hyper_parameter_dict[factor][time_basis]['smape'] = {'rating': 'n/a', **best_mape}
            hyper_parameter_dict[factor][time_basis]['coverage'] = {'rating': 'n/a', **best_coverage}
    
    return hyper_parameter_dict

def save_tuned_hyperparameters(hyper_parameter_dict, suffix=None):
    # Save hyperparameters to a json file
    date = datetime.now().strftime("%Y%m%d")
    
    file_name = f'tuned_macro_hyperparameters_{suffix}_{date}.json'
    
    with open(file_name, 'w') as f:
        json.dump(hyper_parameter_dict, f)
        
    return

def load_latest_tuned_hyperparameters():
    # List all files that begin with "tuned_macro_hyperparameters"
    files = glob.glob('tuned_macro_hyperparameters_*.json')

    # If no files found, return None
    if not files:
        return None

    # Sort files by date
    files.sort(key=os.path.getmtime, reverse=True)

    # Load the most recent file
    latest_file = files[0]
    with open(latest_file, 'r') as f:
        hyper_parameter_dict = json.load(f)

    return hyper_parameter_dict

def load_tuned_hyperparameters(file_name):
    with open(file_name, 'r') as f:
        hyper_parameter_dict = json.load(f)

    return hyper_parameter_dict

initial_window_in_days = 365*6
horizon_in_days = 365 
period_in_days = 180 

# for month to month data, we want to forecast 1 month ahead and do that for N periods
# for quarter to quarter data, we want to forecast 1 quarter ahead and do that for N periods
# for year to year data, we want to forecast 1 year ahead and do that for N periods

# initial window will impact training model

for initial_window_in_days in [365*6, 365*4, 365*3, 365*2, 365]:
    for horizon_in_days in [initial_window_in_days * 0.5, initial_window_in_days * 0.25, initial_window_in_days * 0.125, initial_window_in_days * 0.0625, initial_window_in_days * 0.03125]:
        for period_in_days in [horizon_in_days, horizon_in_days*0.5]:
            print(f'initial_window_in_days: {initial_window_in_days}, period_in_days: {period_in_days}, horizon_in_days: {horizon_in_days}')
            # Tune hyperparameters for macro factors
            hyper_parameter_dict = tune_hyperparameters_for_macro_factors(macro_data_dict, initial_window_in_days, period_in_days, horizon_in_days)

            suffix = f'tuned_hyper_parms_w_init_{initial_window_in_days}_period_{period_in_days}_horizon_{horizon_in_days}'
            save_tuned_hyperparameters(hyper_parameter_dict, suffix)
            
#hyper_parameter_dict = tune_hyperparameters_for_macro_factors(macro_data_dict, initial_window_in_days, period_in_days, horizon_in_days)

#suffix = f'tuned_hyper_parms_w_init_{initial_window_in_days}_period_{period_in_days}_horizon_{horizon_in_days}'
#save_tuned_hyperparameters(hyper_parameter_dict, suffix)

In [ ]:
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from prophet.plot import plot_plotly
import plotly.graph_objs as go
    
import glob

# Define the forecast function
def prophet_forecast(df, periods, data_frequency, hyperparameters):
    
    print(f'prophet_forecast df:\n{df.head()}')
    
    # Check for missing values
    if df['y'].isna().any():
        print("WARNING: The 'y' column contains missing values. These will be dropped.")
        df = df.dropna()

    # Check for non-numeric values
    if df['y'].apply(lambda x: not isinstance(x, (int, float))).any():
        print("WARNING: The 'y' column contains non-numeric values. These cannot be used in the model.")
    
    # Check for zero values
    if (df['y'] == 0).any():
        print("WARNING: The 'y' column contains zero values. These can cause problems for the model.")
    
    # Check for extreme values
    if df['y'].max() > 1e6 or df['y'].min() < -1e6:
        print("WARNING: The 'y' column contains very large or very small values. These can cause problems for the model.")

    # define the model
    model = Prophet(**hyperparameters)
    
    # TODO: for our portfolio forecast, add regressors for each macro factor?
    # model.add_regressor('additional_regressor')
    
    # fit the model
    model.fit(df)
    
    # define the period for which we want a prediction - most data is monthly, use MS for month start
    future = model.make_future_dataframe(periods=periods, freq=data_frequency)
    
    # use the model to make a forecast
    forecast = model.predict(future)
    
    return df, model, forecast

def plot_historical_macro_data_with_forecast(series, forecast, time_series_name, factor_name):
    
    print(f'plot_historical_macro_data_with_forecast {time_series_name} series with factor {factor_name}:\n{series.head()}')
    print(f'plot_historical_macro_data_with_forecast forecast {time_series_name} series with factor {factor_name}:\n{forecast.head()}')
    
    
    # Prophet plot
    fig1 = plot_plotly(model, forecast)
    fig1.update_layout(title=f'{factor_name} Forecast by Prophet')
    
    
    # Your custom plot
    fig2 = go.Figure()
    
    # Add macroeconomic factor trace
    fig2.add_trace(go.Scatter(x=series.index, y=series['y'], mode='lines', name='Historical'))

    # Add forecasted macroeconomic factor trace
    fig2.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast', line=dict(color='red', width=2, dash='dot')))

    # Add confidence interval
    fig2.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill=None, mode='lines', line_color='gray', name='Lower Bound'))
    fig2.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='lines', line_color='gray', name='Upper Bound'))

    # Set axis titles
    fig2.update_yaxes(title_text='Value')
    
    fig2.update_layout(title_text=f'{factor_name} Historical Data with Forecast<br><sup>({time_series_name})</sup>')
    
    return fig1, fig2

initial_window_in_days = 365*6
horizon_in_days = 365 # 6 month forecast horizon
period_in_days = 180 # quarterly training period

# for month to month data, we want to forecast 1 month ahead and do that for N periods
# for quarter to quarter data, we want to forecast 1 quarter ahead and do that for N periods
# for year to year data, we want to forecast 1 year ahead and do that for N periods

def load_latest_tuned_hyperparameters():
    # List all files that begin with "tuned_macro_hyperparameters"
    files = glob.glob('tuned_macro_hyperparameters_*.csv')

    # If no files found, return None
    if not files:
        return None

    # Sort files by date
    files.sort(key=os.path.getmtime, reverse=True)

    # Load the most recent file
    latest_file = files[0]
    df = pd.read_csv(latest_file)

    return df

cur_tuned_params_df = load_latest_tuned_hyperparameters()

years_to_forecast = 10
figs = []
for factor, time_bases in macro_data_dict.items():
    for time_basis, series in time_bases.items():
        
        print(f'Forecasting {factor} from {time_basis} data for series:\n{series.head()}\ndescription:\n{series.describe()}')
        # Get tuned parameters for this time series
        hyperparameters = cur_tuned_params_df.loc[(time_basis, factor)].to_dict()
        print(f'{factor} from {time_basis} leveraging these hyperparameters:\n{hyperparameters}')
        
        df = series.to_frame()
        df.columns = ['y']
        df['ds'] = df.index
        
        if 'Month' in time_basis:
            data_frequency = 'MS'
            periods = years_to_forecast * 12 # 12 months per year
        elif 'Quarter' in time_basis:
            data_frequency = 'QS'
            periods = years_to_forecast * 4 # 4 quarters per year
        elif 'Year' in time_basis:
            data_frequency = 'YS'
            periods = years_to_forecast
        else:
            data_frequency = None
            periods = 365
             
        #horizon = calculate_horizon(df)
        df, model, forecast = prophet_forecast(df, periods, data_frequency, hyperparameters)

        fig2 = plot_historical_macro_data_with_forecast(df, forecast, time_basis, factor)
        
        # Prophet plot
        fig1 = plot_plotly(model, forecast)
        fig1.update_layout(title=f'{factor} Forecast by Prophet')
    
        fig1.show()
        fig2.show()